In [16]:
import re
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.latex import parse_latex
# load from os.env
import os
# New function for statistical comparison:
import random
import numpy as np
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from dotenv import load_dotenv
load_dotenv()
# Set the OpenAI API key from environment variable
# or directly set it here
api_key= os.getenv("OPENAI_API_KEY")
import re
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.latex import parse_latex
import openai
from openai import OpenAI

# Initialize the client using your API key.
client = OpenAI(api_key=api_key)

from sympy.calculus.util import function_range
from sympy import lambdify
from sklearn.metrics import mean_absolute_error


import numpy as np
import sympy as sp
from sympy.calculus.util import function_range
from sympy import lambdify
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

import numpy as np
import sympy as sp
from sympy import lambdify
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr

def stats_test(expr1, expr2, samples=100, default_range=(-10, 10)):
    """
    Compare two SymPy expressions (potentially with multiple variables) via sampling.
    Handles single expressions and equation tuples (compares right-hand sides).
    Calculates:
      - mean absolute error
      - Pearson correlation (+ p-value)
      - R² of pointwise residuals
      - an aggregate similarity_score in [0,1]
    """
    # 1. Unpack equation tuples (compare right-hand sides)
    if isinstance(expr1, tuple) and len(expr1) == 2:
        expr1 = expr1[1]
    if isinstance(expr2, tuple) and len(expr2) == 2:
        expr2 = expr2[1]

    # 2. Identify all variables, sorted for consistent lambdify order
    try:
        variables = sorted(list(expr1.free_symbols.union(expr2.free_symbols)), key=lambda s: s.name)
    except AttributeError as e:
         return {
            "error": f"Could not extract free symbols. Input might not be a valid SymPy expression. Detail: {e}",
            "similarity_score": 0.0,
            "expr1_type": str(type(expr1)),
            "expr2_type": str(type(expr2)),
            "expr1": str(expr1),
            "expr2": str(expr2),
        }

    num_vars = len(variables)

    # 3. Handle pure-constant case
    if num_vars == 0:
        try:
            # Attempt numeric evaluation first
            v1 = float(expr1.evalf() if hasattr(expr1, 'evalf') else expr1)
            v2 = float(expr2.evalf() if hasattr(expr2, 'evalf') else expr2)
            mse = (v1 - v2)**2
            return {
                "type": "constant_comparison",
                "value1": v1,
                "value2": v2,
                "absolute_difference": abs(v1 - v2),
                "mse": mse,
                "are_equal": abs(v1 - v2) < 1e-10,
                "similarity_score": 1.0 / (1.0 + mse),
                "expr1": str(expr1),
                "expr2": str(expr2),
                "variables": []
            }
        except (TypeError, ValueError, AttributeError):
            # Fallback to symbolic comparison for non-numeric constants
            try:
                are_equal_symbolic = sp.simplify(expr1 - expr2) == 0
                return {
                    "type": "symbolic_constant_comparison",
                    "are_equal": are_equal_symbolic,
                    "similarity_score": 1.0 if are_equal_symbolic else 0.0,
                    "expr1": str(expr1),
                    "expr2": str(expr2),
                    "variables": []
                }
            except Exception as e_simp:
                 # If simplification itself fails
                 return {
                    "type": "constant_comparison_failed",
                    "error": f"Symbolic comparison failed: {e_simp}",
                    "similarity_score": 0.0, # Treat as non-equivalent if comparison fails
                    "expr1": str(expr1),
                    "expr2": str(expr2),
                    "variables": []
                 }


    # 4. Lambdify for multi-variable evaluation
    # Add common numpy functions and Abs mapping for compatibility
    func_mapping = {"Abs": np.abs}
    try:
        f1 = lambdify(variables, expr1, modules=["numpy", func_mapping])
        f2 = lambdify(variables, expr2, modules=["numpy", func_mapping])
    except Exception as e:
        return {
            "error": f"Lambdify failed: {e}",
            "variables": [str(v) for v in variables],
            "similarity_score": 0.0,
             "expr1": str(expr1),
             "expr2": str(expr2),
        }

    # 5. Generate multi-dimensional sample points
    # Each row in sample_points corresponds to one sample, columns correspond to variables
    try:
        sample_points = np.random.uniform(default_range[0], default_range[1], size=(samples, num_vars))
    except ValueError as e:
         return {
            "error": f"Failed to generate sample points (check default_range): {e}",
            "variables": [str(v) for v in variables],
            "similarity_score": 0.0,
             "expr1": str(expr1),
             "expr2": str(expr2),
         }

    # 6. Evaluate functions at sample points
    y1_list, y2_list = [], []
    valid_indices = []
    evaluation_errors = 0
    for i, point in enumerate(sample_points):
        try:
            # Pass arguments individually using *
            v1 = complex(f1(*point)).real # Handle potential complex results, take real part
            v2 = complex(f2(*point)).real # Handle potential complex results, take real part

            # Check for NaN or Inf
            if np.isfinite(v1) and np.isfinite(v2):
                y1_list.append(v1)
                y2_list.append(v2)
                valid_indices.append(i)
            else:
                evaluation_errors += 1
        except (TypeError, ValueError, OverflowError, ZeroDivisionError, AttributeError, NameError, KeyError) as e:
             # Catch potential numerical or evaluation errors during function call
            # print(f"Warning: Evaluation skipped for point {point} due to error: {e}") # Optional debug print
            evaluation_errors += 1
            continue # Skip this sample point

    # Check if enough valid points were collected
    if len(y1_list) < 2:
        return {
            "valid_samples": len(y1_list),
            "total_samples": samples,
            "evaluation_errors": evaluation_errors,
            "error": "Insufficient valid data points for statistical analysis after evaluation",
            "variables": [str(v) for v in variables],
            "similarity_score": 0.0,
            "expr1": str(expr1),
            "expr2": str(expr2),
        }

    # Convert lists to numpy arrays
    y1 = np.array(y1_list)
    y2 = np.array(y2_list)
    # valid_sample_points = sample_points[valid_indices] # Keep if needed for debugging

    # 7. Compute statistical metrics
    results = {
        "valid_samples": len(y1),
        "total_samples": samples,
        "evaluation_errors": evaluation_errors,
        "variables": [str(v) for v in variables],
        "sample_range_per_variable": default_range,
        "expr1": str(expr1),
        "expr2": str(expr2),
    }

    # Basic error metrics
    try:
        results["mean_absolute_error"] = mean_absolute_error(y1, y2)
        results["max_absolute_error"] = np.max(np.abs(y1 - y2))
    except Exception as e:
        results["error_mae"] = f"MAE calculation failed: {e}"

    # Pearson correlation - check for constant arrays first
    std_y1 = np.std(y1)
    std_y2 = np.std(y2)
    if std_y1 > 1e-10 and std_y2 > 1e-10: # Check if both have variance
        try:
            corr, pval = pearsonr(y1, y2)
            # Handle potential NaN correlation if variance is extremely small but > 1e-10
            results["correlation"] = corr if np.isfinite(corr) else 1.0 if np.allclose(y1,y2) else 0.0
            results["correlation_p_value"] = pval if np.isfinite(pval) else 0.0 if np.allclose(y1,y2) else 1.0
        except ValueError: # Should be caught by std check, but as safeguard
             results["correlation"] = 1.0 if np.allclose(y1, y2) else 0.0
             results["correlation_p_value"] = 0.0 if np.allclose(y1, y2) else 1.0
        except Exception as e:
            results["error_correlation"] = f"Correlation calculation failed: {e}"
            results["correlation"] = 0.0
            results["correlation_p_value"] = None
    else: # Handle cases where one or both arrays are constant
        # If both are constant and equal, correlation is 1. Otherwise 0.
        are_constant_and_equal = std_y1 <= 1e-10 and std_y2 <= 1e-10 and np.allclose(y1, y2)
        results["correlation"] = 1.0 if are_constant_and_equal else 0.0
        results["correlation_p_value"] = 0.0 if are_constant_and_equal else 1.0


    # R² score
    try:
        # R2 score is undefined if y1 is constant. Check this case.
        if std_y1 > 1e-10:
             results["r2_score"] = r2_score(y1, y2)
        else:
             # If y1 is constant, R2 is 1 if y2 is also constant and equal to y1, else 0.
             results["r2_score"] = 1.0 if std_y2 <= 1e-10 and np.allclose(y1, y2) else 0.0
    except Exception as e:
        results["error_r2"] = f"R2 score calculation failed: {e}"
        results["r2_score"] = 0.0 # Default to 0 in case of error

    # 8. Aggregate similarity score [0,1]
    comps = []
    # Use |correlation| if available and numeric
    corr_val = results.get("correlation")
    if isinstance(corr_val, (float, int)) and np.isfinite(corr_val):
        comps.append(abs(corr_val))

    # Use R² if available and numeric (clamp negative R² to 0 for averaging)
    r2_val = results.get("r2_score")
    if isinstance(r2_val, (float, int)) and np.isfinite(r2_val):
        comps.append(max(0.0, r2_val)) # Clamp R^2 >= 0

    # Inverse-MAE term
    mae_val = results.get("mean_absolute_error")
    if isinstance(mae_val, (float, int)) and np.isfinite(mae_val):
        comps.append(1.0 / (1.0 + mae_val))

    # Calculate mean similarity score if components are available
    if comps:
        results["similarity_score"] = float(np.mean(comps))
    else:
        # Fallback if no components could be calculated
        results["similarity_score"] = 0.0

    # Ensure similarity score is always float and within [0, 1]
    final_score = results.get("similarity_score", 0.0)
    results["similarity_score"] = max(0.0, min(1.0, float(final_score if np.isfinite(final_score) else 0.0)))


    # 9. Final cleanup - remove potentially large data points from results if not needed
    # results.pop("valid_sample_points", None) # Uncomment if sample points were stored

    return results



def normalize_expr(expr):
    """
    Recursively traverse the expression and replace any instance of
    log(expr, E) with log(expr). If expr is a tuple (e.g., representing an
    equation with left and right sides), normalize each element.
    """
    # If expr is a tuple (e.g., (lhs, rhs)), apply normalization to each part.
    if isinstance(expr, tuple):
        return tuple(normalize_expr(e) for e in expr)
    
    # Otherwise, expr should be a Sympy object
    if expr.is_Atom:
        return expr
    new_args = tuple(normalize_expr(arg) for arg in expr.args)
    if expr.func == sp.log and len(new_args) == 2 and new_args[1] == sp.E:
        return sp.log(new_args[0])
    return expr.func(*new_args)

def llm_to_sympy(expr_str: str) -> sp.Expr:
    """
    Convert a mathematical expression string to a Sympy expression using an LLM-based conversion.
    
    This function calls the LLM API and then parses the returned string (which should be a valid 
    Sympy expression) into a Sympy object.
    """
    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """You are a Sympy expression converter. Return only the final Sympy expression 
(e.g., (1/S(2))*log(...) + C). Do not include code fences or additional text. 
No imports, definitions, or explanations. Just one line: the Sympy expression."""},
            {"role": "user", "content": f"""
"You are an agent that converts mathematical expressions to valid Sympy expressions. 
Output only the Sympy expression itself on a single line, with no imports, no extra 
Python code, and no explanations. Use standard Sympy functions like exp, log, sin, 
cos, etc. The result must parse successfully with 'sympy.parse_expr'. 

Convert the following expression to Sympy format: {expr_str}"
"""}
        ],
    )
    output = result.choices[0].message.content.strip()
    
    # Define a local dictionary for standard functions and constants.
    local_dict = {
        'cos': sp.cos,
        'sin': sp.sin,
        'tan': sp.tan,
        'log': sp.log,
        'exp': sp.exp,
        'Derivative': sp.Derivative,
        'Integral': sp.Integral,
        'E': sp.E,
        'e': sp.E,
        'S': sp.S,
        'I': sp.I,
        'pi': sp.pi,
        'sqrt': sp.sqrt,
        'Abs': sp.Abs,

    }
    try:
        sympy_expr = parse_expr(output, local_dict=local_dict)
        return sympy_expr
    except Exception as e:
        raise ValueError(f"Error parsing LLM output:\nOutput: {output}\n{e}")

def preprocess_expression(expr_str: str):
    """
    Preprocess an expression string to produce a Sympy object using an LLM-based conversion.
    
    If the string contains an "=" sign, it is split into left-hand and right-hand parts.
    For each part, the llm_to_sympy function is called to obtain its Sympy expression.
    """
    expr_str = expr_str.strip()
    if '=' in expr_str:
        left_str, right_str = expr_str.split('=', 1)
        # For comparison purposes, we can just return the right-hand side
        return preprocess_expression(right_str.strip())
    else:
        # Call the LLM-based function to convert this expression to a Sympy object.
        return llm_to_sympy(expr_str)


def compare_expressions(expr1, expr2) -> bool:
    """
    Compare two Sympy expressions (or equation tuples). First, normalize both expressions
    (e.g., converting log(expr, E) to log(expr)). Then compare both sides for equations,
    or simply check if the difference is zero for single expressions.
    """
    # Extract right-hand sides if expressions are equations (tuples)
    if isinstance(expr1, tuple) and len(expr1) == 2:
        expr1 = expr1[1]  # Take right-hand side
    if isinstance(expr2, tuple) and len(expr2) == 2:
        expr2 = expr2[1]  # Take right-hand side
    
    # Now both expr1 and expr2 should be single expressions, not tuples
    try:
        return sp.simplify(expr1 - expr2) == 0
    except TypeError:
        # If subtraction fails, try comparing string representations
        return str(expr1) == str(expr2)
def compare_pipeline(expr_str1: str, expr_str2: str) -> bool:
    """
    The complete pipeline that:
      1. Checks if an expression is an equation (using '='), splitting it if necessary.
      2. Calls the LLM-based conversion (llm_to_sympy) to get the Sympy expression.
      3. Normalizes and compares the resulting expressions.
      
    The function prints the parsed expressions and returns whether they are equivalent.
    """
    print("Comparing expressions")
    # Determine if inputs are equations and handle consistently
    is_eq1 = '=' in expr_str1
    is_eq2 = '=' in expr_str2

    try:
        parsed_expr1 = preprocess_expression(expr_str1)
        parsed_expr2 = preprocess_expression(expr_str2)
    except ValueError as e:
        print("Parsing error:", e)
        return False, None, None
    
    # Now we should have compatible expressions (either both tuples or both expressions)
    equivalent = compare_expressions(parsed_expr1, parsed_expr2)
    
    print("Parsed Expression 1:", parsed_expr1)
    print("Parsed Expression 2:", parsed_expr2)
    print("The expressions are equivalent." if equivalent else "The expressions are NOT equivalent.")
    return equivalent, parsed_expr1, parsed_expr2

import pandas as pd
import numpy as np


df = pd.read_csv('onlyfunc.csv',dtype=str,keep_default_na=False)

var  =sp.Symbol('x')
columns_answer = ["deepseek-ai/DeepSeek-V3_answer",'deepseek-ai/DeepSeek-R1_answer',"Qwen/Qwen2.5-72B-Instruct-Turbo_answer","Qwen/QwQ-32B_answer"]
all_results = {}
column_scores = {}

for idx,row in df.iterrows():
    if idx==0:
        continue
    row_results = {}
    try:
        ground_truth = str(row['answer'])
        print("Ground Truth:", ground_truth)

        for col in columns_answer:
            column_expression = str(row[col])
            print("Column Expression:", column_expression)
            try:
                if '=' in column_expression:
                    left_str1, column_expression = column_expression.split('=', 1)
                if '=' in ground_truth:
                    left_str2, ground_truth = ground_truth.split('=', 1)

                result,parsed_expr1,parsed_expr2 = compare_pipeline(column_expression, ground_truth)
                print(result,parsed_expr1,parsed_expr2)
                if result:
                    stats = {"similarity_score": 1.0, "is_equivalent": True}
                else:
                    stats = stats_test(parsed_expr1, parsed_expr2)

                row_results[col] = stats
            except Exception as e:
                print(f"Error processing row {idx}, column {col}: {e}")
                row_results[col] = {"error": str(e), "similarity_score": 0.0}
                # Update column scores with zero for errors
                if col not in column_scores:
                    column_scores[col] = []
                column_scores[col].append(0.0)
                    # Rank columns for this row based on similarity score
        row_rankings = sorted(
                [(col, res["similarity_score"]) for col, res in row_results.items()],
                key=lambda x: x[1],
                reverse=True  # Higher scores are better
        )
                    # Store results for this row
        all_results[idx] = {
                "id":row["Sub Task Id"],
                "ground_truth": ground_truth,
                "column_results": row_results,
                "rankings": row_rankings
            }
        
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        all_results[idx] = {"error": str(e)}



                
avg_scores = {col: np.mean(scores) for col, scores in column_scores.items()}
    # Rank columns based on average similarity score
overall_rankings = sorted(
        [(col, score) for col, score in avg_scores.items()],
        key=lambda x: x[1],
        reverse=True  # Higher scores are better
    )
import json
# save to json file 
with open("all_results.json","w") as f:
    json.dump(all_results , f)


overall_rankings
# expr1 = r'C + log((exp(2*x) - exp(x) + 1)/(exp(2*x) + exp(x) + 1))/2'
# expr2 = r'C + log(Abs((exp(2*x) - exp(x) + 1)/(exp(2*x) + exp(x) + 1)))/2'

# print("Comparing expressions")
# if '=' in expr1:
#         left_str1, expr1 = expr1.split('=', 1)
# if '=' in expr2:
#         left_str2, expr2 = expr2.split('=', 1)



# result,parsed_expr1,parsed_expr2 = compare_pipeline(expr1, expr2)
# print("Result:", "Equivalent" if result else "Not equivalent")
# var = sp.Symbol('x')
# stats = stats_test(parsed_expr1, parsed_expr2)
# print("Statistical test results:",stats)


Ground Truth: g'(2π) = 3/7
Column Expression: \dfrac{3}{7}
Comparing expressions
Parsed Expression 1: 3/7
Parsed Expression 2: 3/7
The expressions are equivalent.
True 3/7 3/7
Column Expression: \dfrac{3}{7}
Comparing expressions
Parsed Expression 1: 3/7
Parsed Expression 2: 3/7
The expressions are equivalent.
True 3/7 3/7
Column Expression: \frac{3}{7}
Comparing expressions
Parsed Expression 1: 3/7
Parsed Expression 2: 3/7
The expressions are equivalent.
True 3/7 3/7
Column Expression: \dfrac{3}{7}
Comparing expressions
Parsed Expression 1: 3/7
Parsed Expression 2: 3/7
The expressions are equivalent.
True 3/7 3/7
Ground Truth: (J - I) = ½ ln( |(e^(2x) - e^x + 1)/(e^(2x) + e^x + 1)| )
Column Expression: \frac{1}{2} \ln \left( \frac{e^{2x} - e^x + 1}{e^{2x} + e^x + 1} \right)
Comparing expressions
Parsed Expression 1: log((exp(2*x) - exp(x) + 1)/(exp(2*x) + exp(x) + 1))/2
Parsed Expression 2: log(Abs((exp(2*x) - exp(x) + 1)/(exp(2*x) + exp(x) + 1)))/2
The expressions are NOT equivalent.

[]

In [ ]:
# def stats_test(expr1, expr2, var, 
#                samples=100, default_range=(-10,10)):
#     # 1. Determine sampling interval
#     try:
#         domain = function_range(expr1 - expr2, var, sp.S.Reals)
#         a, b = float(domain.start), float(domain.end)
#     except:
#         a, b = default_range

#     try:
#         domain = function_range(expr1 - expr2, var, sp.S.Reals)
#         a, b = float(domain.start), float(domain.end)
#         # Ensure a reasonable range size
#         if b - a < 1e-10:
#             a, b = default_range
#     except:
#         a, b = default_range
    

#     # Check if both expressions are constants
#     is_const1 = len(expr1.free_symbols) == 0
#     is_const2 = len(expr2.free_symbols) == 0
    
#     # If both are constants, perform direct numerical comparison
#     if is_const1 and is_const2:
#         val1 = float(expr1)
#         val2 = float(expr2)
#         abs_diff = abs(val1 - val2)
#         mse = abs_diff ** 2
        
#         # Create result dictionary for constant case
#         results = {
#             "type": "constant_comparison",
#             "value1": val1,
#             "value2": val2,
#             "absolute_difference": abs_diff,
#             "mse": mse,
#             "are_equal": abs_diff < 1e-10,  # Numerical equality with tolerance
#             "similarity_score": 1.0 / (1.0 + mse),  # Converts MSE to similarity (0-1 range)
#             "expr1": str(expr1),
#             "expr2": str(expr2)
#         }
#         return results
    
#     # Handle case where only one expression is constant
#     if is_const1:
#         expr1 = sp.Lambda(var, expr1)  # Convert to a constant function
#     if is_const2:
#         expr2 = sp.Lambda(var, expr2)  # Convert to a constant function
    
#     # 1. Determine sampling interval
#     try:
#         domain = function_range(expr1 - expr2, var, sp.S.Reals)
#         a, b = float(domain.start), float(domain.end)
#         # Ensure a reasonable range size
#         if b - a < 1e-10:
#             a, b = default_range
#     except:
#         a, b = default_range
    
#     # 2. Create numerical functions
#     f1 = lambdify(var, expr1, "numpy")
#     f2 = lambdify(var, expr2, "numpy")
    
#     # 3. Generate sample points for x
#     x_values = np.linspace(a, b, samples)
    
#     # 4. Calculate function values, handling potential errors
#     y1_values = []
#     y2_values = []
#     valid_x = []
    
#     for x in x_values:
#         try:
#             y1 = float(f1(x))
#             y2 = float(f2(x))
#             # Only include points where both functions return finite values
#             if (not np.isnan(y1) and not np.isnan(y2) and 
#                 not np.isinf(y1) and not np.isinf(y2)):
#                 y1_values.append(y1)
#                 y2_values.append(y2)
#                 valid_x.append(x)
#         except:
#             continue
    
#     # Convert to numpy arrays for analysis
#     x_array = np.array(valid_x)
#     y1_array = np.array(y1_values)
#     y2_array = np.array(y2_values)
    
#     # Skip analysis if not enough valid points
#     if len(valid_x) < 2:
#         return {
#             "valid_samples": len(valid_x),
#             "error": "Insufficient valid data points for statistical analysis"
#         }
    
#     # 5. Calculate statistical metrics
#     results = {}
    
#     # Number of valid samples
#     results["valid_samples"] = len(valid_x)
    
#     # Calculate absolute and relative differences
#     abs_diff = np.abs(y1_array - y2_array)
#     results["mean_absolute_error"] = np.mean(abs_diff)
#     results["max_absolute_error"] = np.max(abs_diff)
    
#     # Handle division by zero in relative error calculation
#     non_zero_y1 = y1_array != 0
#     if np.any(non_zero_y1):
#         rel_diff = abs_diff[non_zero_y1] / np.abs(y1_array[non_zero_y1])
#         results["mean_relative_error"] = np.mean(rel_diff)
#         results["max_relative_error"] = np.max(rel_diff)
    
#     # Correlation analysis
#     try:
#         correlation, p_value = pearsonr(y1_array, y2_array)
#         results["correlation"] = correlation
#         results["correlation_p_value"] = p_value
#     except:
#         results["correlation"] = "Error calculating correlation"
    
#     # Linear regression analysis
#     try:
#         model = LinearRegression()
#         X = y1_array.reshape(-1, 1)
#         y = y2_array
#         model.fit(X, y)
#         predictions = model.predict(X)
        
#         results["regression_slope"] = model.coef_[0]
#         results["regression_intercept"] = model.intercept_
#         results["r_squared"] = model.score(X, y)
#         results["rmse"] = np.sqrt(mean_squared_error(y, predictions))
        
#         # Perfect match would have slope=1, intercept=0
#         results["slope_deviation"] = abs(1 - model.coef_[0])
#         results["intercept_deviation"] = abs(model.intercept_)
#     except:
#         results["regression_analysis"] = "Error performing regression analysis"
    
#     # 6. Create a simple overall similarity score (0-1)
#     # Weight different factors: correlation, slope deviation, intercept, MAE
#     try:
#         score_components = []
#         if "correlation" in results and isinstance(results["correlation"], (int, float)):
#             score_components.append(abs(results["correlation"]))
#         if "slope_deviation" in results:
#             score_components.append(1 / (1 + results["slope_deviation"]))
#         if "intercept_deviation" in results:
#             score_components.append(1 / (1 + abs(results["intercept_deviation"])))
#         if "mean_absolute_error" in results:
#             score_components.append(1 / (1 + results["mean_absolute_error"]))
        
#         if score_components:
#             results["similarity_score"] = sum(score_components) / len(score_components)
#         else:
#             results["similarity_score"] = "Could not calculate"
#     except:
#         results["similarity_score"] = "Error calculating similarity score"
    
#     # 7. Add expression information
#     results["expr1"] = str(expr1)
#     results["expr2"] = str(expr2)
#     results["sample_range"] = (a, b)
    
#     return results



# def normalize_expr(expr):
#     """
#     Recursively traverse the expression and replace any instance of
#     log(expr, E) with log(expr). If expr is a tuple (e.g., representing an
#     equation with left and right sides), normalize each element.
#     """
#     # If expr is a tuple (e.g., (lhs, rhs)), apply normalization to each part.
#     if isinstance(expr, tuple):
#         return tuple(normalize_expr(e) for e in expr)
    
#     # Otherwise, expr should be a Sympy object
#     if expr.is_Atom:
#         return expr
#     new_args = tuple(normalize_expr(arg) for arg in expr.args)
#     if expr.func == sp.log and len(new_args) == 2 and new_args[1] == sp.E:
#         return sp.log(new_args[0])
#     return expr.func(*new_args)

# def llm_to_sympy(expr_str: str) -> sp.Expr:
#     """
#     Convert a mathematical expression string to a Sympy expression using an LLM-based conversion.
    
#     This function calls the LLM API and then parses the returned string (which should be a valid 
#     Sympy expression) into a Sympy object.
#     """
#     result = client.chat.completions.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "system", "content": """You are a Sympy expression converter. Return only the final Sympy expression 
# (e.g., (1/S(2))*log(...) + C). Do not include code fences or additional text. 
# No imports, definitions, or explanations. Just one line: the Sympy expression."""},
#             {"role": "user", "content": f"""
# "You are an agent that converts mathematical expressions to valid Sympy expressions. 
# Output only the Sympy expression itself on a single line, with no imports, no extra 
# Python code, and no explanations. Use standard Sympy functions like exp, log, sin, 
# cos, etc. The result must parse successfully with 'sympy.parse_expr'. 

# Convert the following expression to Sympy format: {expr_str}"
# """}
#         ],
#     )
#     output = result.choices[0].message.content.strip()
    
#     # Define a local dictionary for standard functions and constants.
#     local_dict = {
#         'cos': sp.cos,
#         'sin': sp.sin,
#         'tan': sp.tan,
#         'log': sp.log,
#         'exp': sp.exp,
#         'Derivative': sp.Derivative,
#         'Integral': sp.Integral,
#         'E': sp.E,
#         'e': sp.E,
#         'C': sp.Symbol('C'),
#         'x': sp.Symbol('x'),
#         'y': sp.Symbol('y'),
#         'z': sp.Symbol('z'),
#         'S': sp.S,
#         'I': sp.I,
#         'pi': sp.pi,
#         'sqrt': sp.sqrt,
#         'Abs': sp.Abs,

#     }
#     try:
#         sympy_expr = parse_expr(output, local_dict=local_dict)
#         return sympy_expr
#     except Exception as e:
#         raise ValueError(f"Error parsing LLM output:\nOutput: {output}\n{e}")

# def preprocess_expression(expr_str: str):
#     """
#     Preprocess an expression string to produce a Sympy object using an LLM-based conversion.
    
#     If the string contains an "=" sign, it is split into left-hand and right-hand parts.
#     For each part, the llm_to_sympy function is called to obtain its Sympy expression.
#     """
#     expr_str = expr_str.strip()
#     if '=' in expr_str:
#         left_str, right_str = expr_str.split('=', 1)
#         left_expr = preprocess_expression(left_str.strip())
#         right_expr = preprocess_expression(right_str.strip())
#         return (left_expr, right_expr)
#     else:
#         # Call the LLM-based function to convert this expression to a Sympy object.
#         return llm_to_sympy(expr_str)

# def compare_expressions(expr1, expr2) -> bool:
#     """
#     Compare two Sympy expressions (or equation tuples). First, normalize both expressions
#     (e.g., converting log(expr, E) to log(expr)). Then compare both sides for equations,
#     or simply check if the difference is zero for single expressions.
#     """
#     norm_expr1 = expr1
#     norm_expr2 = expr2
#     # print("Normalized Expression 1:", norm_expr1)
#     # print("Normalized Expression 2:", norm_expr2)
    
#     if isinstance(norm_expr1, tuple) and isinstance(norm_expr2, tuple):
#         left_equal = sp.simplify(norm_expr1[0] - norm_expr2[0]) == 0
#         right_equal = sp.simplify(norm_expr1[1] - norm_expr2[1]) == 0
#         return left_equal and right_equal
#     else:
#         return sp.simplify(norm_expr1 - norm_expr2) == 0
# def compare_pipeline(expr_str1: str, expr_str2: str) -> bool:
#     """
#     The complete pipeline that:
#       1. Checks if an expression is an equation (using '='), splitting it if necessary.
#       2. Calls the LLM-based conversion (llm_to_sympy) to get the Sympy expression.
#       3. Normalizes and compares the resulting expressions.
      
#     The function prints the parsed expressions and returns whether they are equivalent.
#     """
#     print("Comparing expressions")
#     if '=' in expr_str1:
#             left_str1, expr_str1 = expr_str1.split('=', 1)
#     if '=' in expr_str2:
#             left_str2, expr_str2 = expr_str2.split('=', 1)

#     try:
#         parsed_expr1 = preprocess_expression(expr_str1)
#         parsed_expr2 = preprocess_expression(expr_str2)
#     except ValueError as e:
#         print("Parsing error:", e)
#         return False
    
#     equivalent = compare_expressions(parsed_expr1, parsed_expr2)
    
#     print("Parsed Expression 1:", parsed_expr1)
#     print("Parsed Expression 2:", parsed_expr2)
#     print("The expressions are equivalent." if equivalent else "The expressions are NOT equivalent.")
#     return equivalent, parsed_expr1, parsed_expr2


# # Example with plain text vs. LaTeX. They should be equivalent.
# expr1 = r'x = (2√5 - 1) / 2'
# expr2 = r'x = \dfrac{1 + \sqrt{2} + \sqrt{2\sqrt{2} - 1}}{2}'

# print("Comparing expressions")
# if '=' in expr1:
#         left_str1, expr1 = expr1.split('=', 1)
# if '=' in expr2:
#         left_str2, expr2 = expr2.split('=', 1)



# result,parsed_expr1,parsed_expr2 = compare_pipeline(expr1, expr2)
# print("Result:", "Equivalent" if result else "Not equivalent")
# var = sp.Symbol('x')
# stats = stats_test(parsed_expr1, parsed_expr2, var)
# print("Statistical test results:",stats)

Comparing expressions
Comparing expressions
Parsed Expression 1: -1/2 + sqrt(5)
Parsed Expression 2: 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2
The expressions are NOT equivalent.
Result: Not equivalent
Statistical test results: {'type': 'constant_comparison', 'value1': 1.7360679774997898, 'value2': 1.883203505913526, 'absolute_difference': 0.14713552841373612, 'mse': 0.021648863721589348, 'are_equal': False, 'similarity_score': 0.9788098783345891, 'expr1': '-1/2 + sqrt(5)', 'expr2': '1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2'}


### using the csv

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('together_deepseek - Sheet3.csv',dtype=str,keep_default_na=False)

var  =sp.Symbol('x')
columns_answer = ["deepseek-ai/DeepSeek-V3_answer",'deepseek-ai/DeepSeek-R1_answer',"Qwen/Qwen2.5-72B-Instruct-Turbo_answer","Qwen/QwQ-32B_answer"]
all_results = {}
column_scores = {}

for idx,row in df.iterrows():
    if idx<2:
        continue
    row_results = {}
    try:
        ground_truth = str(row['answer'])
        print("Ground Truth:", ground_truth)

        for col in columns_answer:
            column_expression = str(row[col])
            print("Column Expression:", column_expression)
            try:
                result,parsed_expr1,parsed_expr2 = compare_pipeline(column_expression, ground_truth)
                print(result,parsed_expr1,parsed_expr2)
                if result:
                    stats = {"similarity_score": 1.0, "is_equivalent": True}
                else:
                    stats = stats_test(parsed_expr1, parsed_expr2, var)

                row_results[col] = stats
            except Exception as e:
                print(f"Error processing row {idx}, column {col}: {e}")
                row_results[col] = {"error": str(e), "similarity_score": 0.0}
                # Update column scores with zero for errors
                if col not in column_scores:
                    column_scores[col] = []
                column_scores[col].append(0.0)
                    # Rank columns for this row based on similarity score
        row_rankings = sorted(
                [(col, res["similarity_score"]) for col, res in row_results.items()],
                key=lambda x: x[1],
                reverse=True  # Higher scores are better
        )
                    # Store results for this row
        all_results[idx] = {
                "id":row["Sub Task Id"],
                "ground_truth": ground_truth,
                "column_results": row_results,
                "rankings": row_rankings
            }
        
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        all_results[idx] = {"error": str(e)}


                
avg_scores = {col: np.mean(scores) for col, scores in column_scores.items()}
    # Rank columns based on average similarity score
overall_rankings = sorted(
        [(col, score) for col, score in avg_scores.items()],
        key=lambda x: x[1],
        reverse=True  # Higher scores are better
    )
import json
# save to json file 
with open("all_results.json","w") as f:
    json.dump(all_results , f)

Ground Truth: x = -1
Column Expression: -1
Comparing expressions
Parsed Expression 1: -1
Parsed Expression 2: -1
The expressions are equivalent.
True -1 -1
Column Expression: -1
Comparing expressions
Parsed Expression 1: -1
Parsed Expression 2: -1
The expressions are equivalent.
True -1 -1
Column Expression: \pm \sqrt{3}
Comparing expressions
Parsed Expression 1: 1/6
Parsed Expression 2: -1
The expressions are NOT equivalent.
False 1/6 -1
Column Expression: -1
Comparing expressions
Parsed Expression 1: -1
Parsed Expression 2: -1
The expressions are equivalent.
True -1 -1
Ground Truth: a + b + c = 0  
Column Expression: a + b + c = 0
Comparing expressions
Parsed Expression 1: 0
Parsed Expression 2: 0
The expressions are equivalent.
True 0 0
Column Expression: a + b + c = 0
Comparing expressions
Parsed Expression 1: 0
Parsed Expression 2: 0
The expressions are equivalent.
True 0 0
Column Expression: a + b + c = 0
Comparing expressions
Parsed Expression 1: 0
Parsed Expression 2: 0
The exp

<lambdifygenerated-21>:2: RuntimeWarning: invalid value encountered in log
  return C + (1/2)*log(x + sqrt(x**2 - 1))
<lambdifygenerated-21>:2: RuntimeWarning: invalid value encountered in sqrt
  return C + (1/2)*log(x + sqrt(x**2 - 1))


Error processing row 3: 'similarity_score'
Ground Truth: x = (2√5 - 1) / 2
Column Expression: \dfrac{1 + \sqrt{2} + \sqrt{2\sqrt{2} - 1}}{2}
Comparing expressions
Parsed Expression 1: 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2
Parsed Expression 2: -1/2 + sqrt(5)
The expressions are NOT equivalent.
False 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2 -1/2 + sqrt(5)
Column Expression: \dfrac{1 + \sqrt{2} + \sqrt{2\sqrt{2} - 1}}{2}
Comparing expressions
Parsed Expression 1: 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2
Parsed Expression 2: -1/2 + sqrt(5)
The expressions are NOT equivalent.
False 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2 -1/2 + sqrt(5)
Column Expression: 2
Comparing expressions
Parsed Expression 1: 2
Parsed Expression 2: -1/2 + sqrt(5)
The expressions are NOT equivalent.
False 2 -1/2 + sqrt(5)
Column Expression: \dfrac{1 + \sqrt{2} + \sqrt{2\sqrt{2} - 1}}{2}
Comparing expressions
Parsed Expression 1: 1/2 + sqrt(-1 + 2*sqrt(2))/2 + sqrt(2)/2
Parsed Expression 2: -1/2 + sqrt(5)
The expres

In [ ]:
# open the all_results.json file and for each question give the rank of models based on similarity score
import json
with open("alll_results.json","r") as f:
    all_results = json.load(f)

# for every id in all_results make a dict with key id and mapping to a ranked list of model and their similarity scores 
ranked_list = {}
for key,row in all_results:
    #get similarity score 
    columns_results = row["column_results"]
    models_score = []
    for model,answer in columns_results:
        models_score.append((model,answer["similarity_score"]))

    models_score = sorted(models_score, key=lambda x: x[1], reverse=True)
    ranked_list[row['id']] = models_score

print(ranked_list)
    

In [21]:
all_results['12']["rankings"]

[['deepseek-ai/DeepSeek-V3_answer', 0.0],
 ['deepseek-ai/DeepSeek-R1_answer', 0.0],
 ['Qwen/Qwen2.5-72B-Instruct-Turbo_answer', 0.0],
 ['Qwen/QwQ-32B_answer', 0.0]]